# Code and visualization for the challenge

In [40]:
#import lib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC


In [41]:
training_v2 = pd.read_csv("Data/training_v2.csv")
test = pd.read_csv("Data/unlabeled.csv")
y = training_v2['hospital_death']
X = training_v2
X = training_v2.drop('hospital_death',axis = 1)

In [42]:
descrip_dict = pd.read_csv("Data/WiDS Datathon 2020 Dictionary.csv",index_col = ['Variable Name'])

### Missing data
Tina

Summary of missing data

Select and fill in important variables that miss a lot of data



In [43]:
#import chart_studio.plotly as py
import missingno as msno 
import plotly.express as px

In [44]:
X.describe()

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,...,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,91713.000000,91713.000000,91713.000000,87485.000000,88284.000000,91713.000000,90379.000000,91713.000000,91713.000000,91713.0,...,83766.000000,83766.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000
mean,65606.079280,65537.131464,105.669262,62.309516,29.185818,0.183736,169.641588,508.357692,0.835766,0.0,...,0.086787,0.043955,0.000857,0.015693,0.225192,0.012989,0.026165,0.007066,0.004132,0.020638
std,37795.088538,37811.252183,62.854406,16.775119,8.275142,0.387271,10.795378,228.989661,2.487756,0.0,...,0.247569,0.217341,0.029265,0.124284,0.417711,0.113229,0.159628,0.083763,0.064148,0.142169
min,1.000000,1.000000,2.000000,16.000000,14.844926,0.000000,137.200000,82.000000,-24.947222,0.0,...,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32852.000000,32830.000000,47.000000,52.000000,23.641975,0.000000,162.500000,369.000000,0.035417,0.0,...,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,65665.000000,65413.000000,109.000000,65.000000,27.654655,0.000000,170.100000,504.000000,0.138889,0.0,...,0.050000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,98342.000000,98298.000000,161.000000,75.000000,32.930206,0.000000,177.800000,679.000000,0.409028,0.0,...,0.130000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,131051.000000,131051.000000,204.000000,89.000000,67.814990,1.000000,195.590000,927.000000,159.090972,0.0,...,0.990000,0.970000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [45]:
total_patient_data = X.index.nunique()
high_miss_var = []
miss_perct = []
threshold = .50 # variables that have missing 60% 

for var in X.columns:
    count = 0 
    for value in X[var].values:
        if pd.isnull(value):    
            count = count + 1     
    miss_rate = (count/total_patient_data)
    if miss_rate >= threshold:
        high_miss_var.append(var)
        miss_perct.append(miss_rate)
missing_stat = pd.DataFrame(miss_perct, columns = ['miss_rate'], index = high_miss_var)
missing_stat

,miss_rate
albumin_apache,0.592926
bilirubin_apache,0.633869
fio2_apache,0.772715
paco2_apache,0.772715
paco2_for_ph_apache,0.772715
...,...
h1_arterial_ph_min,0.833295
h1_arterial_po2_max,0.828072
h1_arterial_po2_min,0.828072
h1_pao2fio2ratio_max,0.874413


Variables that have high percentages of missing values are illustrated below

In [57]:
fig = px.bar(missing_stat, x=missing_stat.index, y=missing_stat.miss_rate, 
             title = "Visualization of Variables with Highest Missing Rate")
fig.show()

*Compute augmented matrix by incorporating missing values as an indicator*

reference:
- [A New Insight Into Missing Data in Intensive Care Unit Patient Profiles: Observational Study
](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6329436/)

In [32]:
#auxiliary matrix (containing the missingness indicators: 0-present, 1-absent)
aux_mask= pd.isnull(X)
augmented_mask = X
for m_var in list(missing_stat.index):
    augmented_mask[m_var] = aux_mask[m_var]

In [55]:
augmented_mask

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,68.0,22.730000,0,Caucasian,M,180.3,Floor,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,77.0,27.420000,0,Caucasian,F,160.0,Floor,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,25.0,31.950000,0,Caucasian,F,172.7,Emergency Department,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,81.0,22.640000,1,Caucasian,F,165.1,Operating Room,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,19.0,NaN,0,Caucasian,M,188.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91708,91592,78108,30,75.0,23.060250,0,Caucasian,M,177.8,Acute Care/Floor,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,Sepsis,Cardiovascular
91709,66119,13486,121,56.0,47.179671,0,Caucasian,F,183.0,Emergency Department,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
91710,8981,58179,195,48.0,27.236914,0,Caucasian,M,170.2,Emergency Department,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
91711,33776,120598,66,NaN,23.297481,0,Caucasian,F,154.9,Emergency Department,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory


### Feature selection
Yu

remove Collinear (highly correlated) features

(Optional)remove Features with zero importance in a tree-based model


In [18]:
# Remove the colinear features
# threshold for the correlation
threshold = 0.9

# Absolute value correlation matrix
corr_matrix = X.corr().abs()
corr_matrix.head()

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,...,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
encounter_id,1.000000,0.009575,0.004532,0.003990,0.001068,0.002036,0.005400,0.000992,0.000570,NaN,...,0.000513,0.000344,0.001907,0.007601,0.003402,0.000972,0.001707,0.002172,0.001176,0.004646
patient_id,0.009575,1.000000,0.007075,0.006343,0.001380,0.001387,0.002902,0.001770,0.004412,NaN,...,0.003735,0.001625,0.002426,0.001307,0.000434,0.001667,0.000548,0.000844,0.002333,0.003065
hospital_id,0.004532,0.007075,1.000000,0.008673,0.012874,0.052123,0.027895,0.004526,0.001285,NaN,...,0.006993,0.000283,0.004947,0.002962,0.011978,0.001362,0.000146,0.002992,0.002588,0.004730
age,0.003990,0.006343,0.008673,1.000000,0.087077,0.067320,0.109937,0.024257,0.049872,NaN,...,0.143167,0.076275,0.029477,0.028065,0.077908,0.020061,0.025007,0.030310,0.023335,0.025924
bmi,0.001068,0.001380,0.012874,0.087077,1.000000,0.015921,0.056316,0.001403,0.001531,NaN,...,0.033546,0.013796,0.020434,0.002377,0.172943,0.001855,0.031144,0.013375,0.010017,0.043380


In [19]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print('There are %d columns to be removed.' % (len(to_drop)))
print("These are columns to be removed", to_drop)
#Drop the columns with high correlations
X = X.drop(columns = to_drop)

There are 59 columns to be removed.
These are columns to be removed ['apache_post_operative', 'paco2_for_ph_apache', 'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min', 'd1_mbp_noninvasive_max', 'd1_mbp_noninvasive_min', 'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'h1_diasbp_noninvasive_max', 'h1_diasbp_noninvasive_min', 'h1_mbp_noninvasive_max', 'h1_mbp_noninvasive_min', 'h1_sysbp_noninvasive_max', 'h1_sysbp_noninvasive_min', 'd1_albumin_max', 'd1_albumin_min', 'd1_bilirubin_max', 'd1_bilirubin_min', 'd1_bun_max', 'd1_bun_min', 'd1_creatinine_max', 'd1_creatinine_min', 'd1_glucose_max', 'd1_hemaglobin_min', 'd1_hematocrit_max', 'd1_hematocrit_min', 'd1_platelets_min', 'd1_sodium_min', 'd1_wbc_max', 'd1_wbc_min', 'h1_albumin_min', 'h1_bilirubin_max', 'h1_bilirubin_min', 'h1_bun_max', 'h1_bun_min', 'h1_calcium_min', 'h1_creatinine_max', 'h1_creatinine_min', 'h1_glucose_min', 'h1_hco3_max', 'h1_hco3_min', 'h1_hemaglobin_max', 'h1_hemaglobin_min', 'h1_hematocrit_max', 'h

### Splitting data


In [20]:
X_train, X_test ,y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


### Data imputation
Jenny

For features that miss not a lot of data, try interpolation or matrix factorization(do it separately for train and test set)

### Features preprocessing
Anyone

convert features into their correct form, categories to categories, numeric number to real number


### Models
pick whichever you want
### Logistic regression
Jewel

### SVM

In [23]:
clf = SVC(kernel = 'poly')    # try 'linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ 
clf.fit(X_train, y_train)                  #Train the model using the training sets
y_pred = clf.predict(X_test)                #Predict the response for test dataset
print(precision_recall_fscore_support(y_test, y_pred, average='binary'))

ValueError: could not convert string to float: 'Neurologic'

### Random forest

In [22]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
print(precision_recall_fscore_support(y_test, predictions, average='binary'))

ValueError: could not convert string to float: 'Other/Unknown'

### Boosting methods
Yu

### Neural Net